In [1]:
import torch
import numpy as np
import pandas as pd
import os

os.chdir("../")

%load_ext autoreload
%autoreload 2

In [20]:
from src.data.preprocessing import compute_class_weight

test_data = torch.randint(0, 10, (1000,))
weights = compute_class_weight(test_data, 20)
weights

tensor([1.0000, 0.6864, 0.9000, 0.8901, 0.7105, 0.7297, 0.7788, 0.9000, 0.8020,
        0.8100, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000])

In [26]:
from src.models.components.heads.utrseg import UTRSegDecoder

trseg = UTRSegDecoder(768, 20, (3, 8, 8), [512, 256, 128, 64])
trseg

UTRSegDecoder(
  (clpse): ModuleList(
    (0-3): 4 x ReduceTemporalFeature(
      (depth_collapse): Sequential(
        (0): Conv3d(768, 768, kernel_size=(3, 1, 1), stride=(1, 1, 1))
        (1): BatchNorm3d(768, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
      (pred): Identity()
    )
  )
  (deconv): ModuleList(
    (0): Sequential(
      (0): TRDeconv(
        (deconv): Sequential(
          (0): ConvTranspose2d(768, 512, kernel_size=(2, 2), stride=(2, 2))
          (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
      )
    )
    (1): Sequential(
      (0): TRDeconv(
        (deconv): Sequential(
          (0): ConvTranspose2d(768, 512, kernel_size=(2, 2), stride=(2, 2))
          (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
      )
      (1): TRDeconv(
     

In [30]:
input_test = [
    torch.rand(1, 192, 768),
    torch.rand(1, 192, 768),
    torch.rand(1, 192, 768),
    torch.rand(1, 192, 768)
]
output_test = trseg(input_test)
output_test.shape

torch.Size([1, 20, 128, 128])

In [9]:
from src.models.components.heads.unetq import TemporalAttentionDeconv

head = TemporalAttentionDeconv(768, 20, (3, 8, 8), [512, 256, 128])
head

TemporalAttentionDeconv(
  (merge_attn): MergeAttentionBlock(
    (attn): SelfAttentionLayer(
      (attn): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
      )
    )
    (merge): MergeLayer(
      (merge): Sequential(
        (0): Conv3d(768, 768, kernel_size=(3, 1, 1), stride=(1, 1, 1))
        (1): BatchNorm3d(768, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
      (pred): Conv2d(768, 768, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    )
  )
  (deconv): ConvTranspose2d(768, 512, kernel_size=(2, 2), stride=(2, 2))
  (bottleneck): ModuleList(
    (0): DeconvBottleneck(
      (deconv): Sequential(
        (0): Sequential(
          (0): ConvTranspose2d(768, 512, kernel_size=(2, 2), stride=(2, 2))
          (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
  

In [10]:
test_input = [
    torch.rand(1, 768, 192),
    torch.rand(1, 768, 192),
    torch.rand(1, 768, 192),
    torch.rand(1, 768, 192),
]
head(test_input).shape

torch.Size([1, 20, 128, 128])

In [ ]:
from torch import nn
from timm.models.swin_transformer import (
    SwinTransformerBlock
)
from typing import Literal

class PatchExpanding(nn.Module):
    def __init__(
            self,
            dim,
            input_resolution,
            dim_scale=2, # type: Literal[0, 2]
            norm_layer=nn.LayerNorm,
            norm_kwargs=None
            ):
        super().__init__()
        self.dim = dim
        self.input_resolution = input_resolution
        self.expand = nn.Linear(dim, dim * dim_scale, bias=False) if dim_scale == 2 else nn.Identity()
        self.norm = norm_layer(dim * dim_scale, **({} if norm_kwargs is None else norm_kwargs))
    
    def forward(self, x):
        x = self.expand(x)
        B, L, C = x.shape
        


class SwinBlock(nn.Module):
    def __init__(
            self,
            dim,
            input_resolution,
            shift_size
            ):
        super().__init__()
        self.ssbq = nn.Sequential(
            SwinTransformerBlock(
                dim=dim,
                input_resolution=input_resolution,
                num_heads=1,
                mlp_ratio=4.
            ),
            SwinTransformerBlock(
                dim=dim,
                input_resolution=input_resolution,
                num_heads=1,
                shift_size=shift_size,
                mlp_ratio=4.
            )
            )
        
    def forward(self, x):
        return self.ssbq(x)



In [37]:
from src.data.functional import random_rotation

test = torch.tensor([[
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9]
    ]])
test.shape

random_rotation(test)

tensor([[[3, 6, 9],
         [2, 5, 8],
         [1, 4, 7]]])

In [21]:
from src.data.transforms import CombinedRandomTransform


def test_a(data: ...):
    print(f"test a data: {data}")

def test_b(data: ...):
    print(f"test b data: {data}")

transform = CombinedRandomTransform(test_a, test_b)
print(transform(torch.tensor([1])))
print(transform(torch.tensor([2])))
print(transform(torch.tensor([3])))
print(transform(torch.tensor([4])))

test a data: tensor([1])
None
test b data: tensor([2])
None
(tensor([3]),)
(tensor([4]),)


In [45]:
from src.data.transforms import loose_bind_transforms

type(loose_bind_transforms(test_a))

function

In [38]:
from src.utils.scripting import loose_bind_kwargs

@loose_bind_kwargs(True)
def fn(a, b, c):
    return a + b + c

kwarg = {
    "a": 1,
    "b": 2,
    "c": 3,
    "d": 4,
}
fn(**kwarg)


[rank: 0] Discarded kwarg: {'d'}


6

In [12]:
from terratorch.models.encoder_decoder_factory import EncoderDecoderFactory
from terratorch.datasets import HLSBands
from terratorch.models.prithvi_model_factory import PrithviModelFactory

factory = EncoderDecoderFactory()
model = factory.build_model(
    task="segmentation",
    backbone="prithvi_vit_100",
    decoder="FCNDecoder",
    num_classes=20,
    backbone_in_chans="backbone_in_chans",
    backbone_pretrained=True,
    backbone_num_frames=3,
    backbone_bands=[
        HLSBands.BLUE,
        HLSBands.GREEN,
        HLSBands.RED,
        HLSBands.NIR_NARROW,
        HLSBands.SWIR_1,
        HLSBands.SWIR_2
    ],
    necks=None
)
model

INFO:timm.models._builder:Loading pretrained weights from Hugging Face hub (('ibm-nasa-geospatial/Prithvi-100M', 'Prithvi_EO_V1_100M.pt'))
c:\Users\EMC\anaconda3\envs\sits-crop-recognition\Lib\site-packages\timm\models\_hub.py:190: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control o

PixelWiseModel(
  (encoder): TimmBackboneWrapper(
    (_timm_module): PrithviViT(
      (patch_embed): PatchEmbed(
        (proj): Conv3d(6, 768, kernel_size=(1, 16, 16), stride=(1, 16, 16))
        (norm): Identity()
      )
      (blocks): ModuleList(
        (0-11): 12 x Block(
          (norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): Attention(
            (qkv): Linear(in_features=768, out_features=2304, bias=True)
            (q_norm): Identity()
            (k_norm): Identity()
            (attn_drop): Dropout(p=0.0, inplace=False)
            (proj): Linear(in_features=768, out_features=768, bias=True)
            (proj_drop): Dropout(p=0.0, inplace=False)
          )
          (ls1): Identity()
          (drop_path1): Identity()
          (norm2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): Mlp(
            (fc1): Linear(in_features=768, out_features=3072, bias=True)
            (act): GELU(approximate='none')
       